In [3]:
#!pip install transformers torch torchvision torchaudio peft accelerate sentencepiece gguf
#!pip install -U git+https://github.com/huggingface/peft.git

In [ ]:
# python llama.cpp/convert-hf-to-gguf.py unload --outfile model/mistral-rag-instruct.gguf --outtype f16
#./llama.cpp/llama-quantize ./model/mistral-rag-instruct.gguf ./model/mistral-rag-instruct.Q4.gguf Q4_K_M

# Merging PEFT (LoRA) Adapter into base model and saving it to disk

In [1]:
import torch; torch.version.cuda

'12.1'

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

# checkpoints/raft/checkpoint-425
# checkpoints/raft-v2/checkpoint-250
base_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.3", torch_dtype=torch.bfloat16, device_map={"": "cpu"})
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.3")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
model = PeftModel.from_pretrained(base_model, "checkpoints/raft-v2/checkpoint-250", device_map={"": "cpu"})

In [3]:
model.config.to_dict()["torch_dtype"]

'bfloat16'

In [4]:
model = model.merge_and_unload(safe_merge=True, progressbar=True)

Unloading and merging model:   0%|          | 0/679 [00:00<?, ?it/s]

Unloading and merging model: 100%|██████████| 679/679 [00:16<00:00, 40.26it/s]


In [5]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
     

In [6]:
state_dict= model.state_dict
state_dict()

OrderedDict([('model.embed_tokens.weight',
              tensor([[-2.0336e-36,  3.3208e-37, -1.5517e-35,  ..., -4.9371e-36,
                       -7.9934e-36, -5.9480e-36],
                      [-4.4250e-03, -2.0981e-05, -5.6458e-03,  ..., -6.5804e-05,
                       -1.0300e-03,  8.5354e-05],
                      [-1.4572e-03,  8.6975e-04,  1.7357e-04,  ...,  1.0910e-03,
                        3.1471e-04,  3.0994e-05],
                      ...,
                      [ 4.4556e-03, -1.4877e-03,  3.8338e-04,  ..., -5.3942e-06,
                        5.5237e-03,  1.2436e-03],
                      [-5.3024e-04,  3.3264e-03,  3.2501e-03,  ...,  1.1139e-03,
                        1.4572e-03,  9.0408e-04],
                      [-3.8147e-03, -6.0654e-04, -5.8594e-03,  ...,  4.2419e-03,
                       -4.3106e-04, -2.5024e-03]], dtype=torch.bfloat16)),
             ('model.layers.0.self_attn.q_proj.weight',
              tensor([[ 9.2983e-06,  7.7057e-04, -3.6240e-05,  

In [8]:
model.save_pretrained("model/raftv2", state_dict=state_dict())
#tokenizer.save_pretrained("unload")

In [2]:
tokenizer.save_pretrained("model/raftv2")

('model/raft/tokenizer_config.json',
 'model/raft/special_tokens_map.json',
 'model/raft/tokenizer.model',
 'model/raft/added_tokens.json',
 'model/raft/tokenizer.json')

In [2]:
from transformers import AutoTokenizer
import torch

In [5]:
# checkpoints/raft/checkpoint-425
# checkpoints/raft-v2/checkpoint-250
tokenizer = AutoTokenizer.from_pretrained("checkpoints/raft-v2/checkpoint-250")
tokenizer

LlamaTokenizerFast(name_or_path='checkpoints/raft-v2/checkpoint-250', vocab_size=32768, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[/INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[TOOL_CALLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("[AVAILABLE_TOOLS]",

In [7]:
tokenizer.save_pretrained("model/raftv2")

('model/raftv2/tokenizer_config.json',
 'model/raftv2/special_tokens_map.json',
 'model/raftv2/tokenizer.model',
 'model/raftv2/added_tokens.json',
 'model/raftv2/tokenizer.json')

In [6]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.3")
tokenizer.pad_token = tokenizer.unk_token
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.3', vocab_size=32768, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[/INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[TOOL_CALLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("[

## save to awq quantization

In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model = AutoAWQForCausalLM.from_pretrained( 'unload', device_map="auto")
tokenizer = AutoTokenizer.from_pretrained( 'unload' )

quant_path = 'merged_model-AWQ'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 6, "version": "GEMM" }


model.quantize( tokenizer, quant_config = quant_config, export_compatible=True )
model.save_quantized( quant_path )
tokenizer.save_pretrained( quant_path )

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
AWQ: 100%|██████████| 32/32 [17:05<00:00, 32.05s/it]
Note that `shard_checkpoint` is deprecated and will be removed in v4.44. We recommend you using split_torch_state_dict_into_shards from huggingface_hub library


('merged_model-AWQ/tokenizer_config.json',
 'merged_model-AWQ/special_tokens_map.json',
 'merged_model-AWQ/tokenizer.model',
 'merged_model-AWQ/added_tokens.json',
 'merged_model-AWQ/tokenizer.json')

In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

quant_path = 'merged_model-AWQ'

awq_model = AutoAWQForCausalLM.from_quantized(quant_path, fuse_layers=True)
tokenizer = AutoTokenizer.from_pretrained(quant_path) 

Fusing layers...: 100%|██████████| 32/32 [00:00<00:00, 110.74it/s]


In [4]:
from transformers import AutoModelForCausalLM
import torch

base_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.3", torch_dtype=torch.bfloat16, device_map={"": "cpu"})


model = awq_model.merge_and_unload(safe_merge=True, progressbar=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

AttributeError: 'MistralAWQForCausalLM' object has no attribute 'merge_and_unload'

In [6]:
import torch

data = [{"page_content": "\n\u00a7 4 Sprachkenntnisse\n(1) 1Neben den allgemeinen Zugangsvoraussetzungen (\u00a7 59 LHG) sind f\u00fcr die in \u00a7 1 Abs. 1\nS. 1 genannten Studieng\u00e4nge deutsche Sprach kenntnisse nachzuweisen. 2Diese k\u00f6nnen\ndurch eine deutsche Hochschulzugangsberechtigung (u. a. erfolgreich abgeschlossenes\ngrundst\u00e4ndiges Hochschulstudium) nachgewiesen werden. 3Ferner kann der\nSprachnachweis durch die Vorlage eines der folgenden Dokumente erbracht werden:\n1. Feststellungspr\u00fcfung f\u00fcr ein Bachelorstudium durch Vorlage der Zugangsberechtigung\ndes Studienkollegs an der Hochschule Konstanz,\n2. Test Deutsch als Fremdsprache (TestDaF), sofern im Durchschnitt mindestens die\nStufe TDN 4 erreicht wurde,   Seite 5 von 43 3. Deutsche Sprachpr\u00fcfung f\u00fcr den Hochschulzugang (DSH), sofern die DSH mit\nmindestens der Stufe DSH -2 abgeschlossen wurde,\n4. \u201eTelc Deutsch C1 Hochschule\u201c\noder eine \u00e4quivalente Sprachpr\u00fcfung gem\u00e4\u00df der Rahmenordnung \u00fcber Deutsche\nSprachpr\u00fcfungen f\u00fcr das Studium an deutschen Hochschulen (RO -DT). 4Auf den Nachweis\neiner deutschen Sprachpr\u00fcfung kann bei Bewerber innen und Bewerbern im besonders\nbegr\u00fcndeten Einzelfall verzichtet werden, insbesondere wenn sie die deutsche\nStaatsangeh\u00f6rigkeit besitzen.\n(2) 1Sprachnachweise f\u00fcr den gew\u00e4hl ten Studiengang, die durch die Bewerberin oder den\nBewerber bis zum Bewerbungsschluss nicht vorgelegt werden k\u00f6nnen, k\u00f6nnen bis zum\nVorlesungsbeginn des Semesters gem\u00e4\u00df Terminplan der Hochschule Konstanz, f\u00fcr das  der\nAntrag auf Zulassung gestellt wurde, nachgereicht werden. 2Die Zulassung erfolgt in diesem\nFall gem \u00e4\u00df \u00a7 6  Abs. 5  unter Vorbehalt .\n(3) 1F\u00fcr Zeitstudierende gelten die Regelungen in \u00a7 10 Zulassungs - und\nImmatrikulationsordnung (ZIO) der Hochschule Konstanz.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 21b Mechatronik (MME) Berufsbegleitendes Studium\n(1) Studiengangspezifische Zugangsvoraussetzungen gem\u00e4\u00df \u00a7 5  Abs. 1\nZugangsvoraussetzungen f\u00fcr den Masterstudiengang Mechatronik sind:\n1. Ein mit der Note 2,9 oder besser abgeschlossenes grundst\u00e4ndiges Hochschulstudium\ngem\u00e4\u00df \u00a7 5 Abs. 1 Nr. 1 in einem Studiengang der Fachrichtungen Systemtechnik,\nMaschinenbau, Elektrotechnik, Fahrzeugtechnik, Mechatronik, Feinwerktechnik oder einer\nverwandten Fachrichtung.\n2. Englischkenntnisse, \u00e4quivalent z u Niveau- Stufe B1 des Europ\u00e4ischen Referenzrahmens\nf\u00fcr das Lernen und Lehren von Fremdsprachen. Als \u00e4quivalent zu einem Zertifikat \u00fcber die\nNiveau -Stufe B1 gelten insbesondere folgende Nachweise:\nI. das Schulabschlusszeugnis, aus dem der Besuch des Englischunterrichts bis zum\nErreichen des mittleren Bildungsabschlusses (10. Klasse) bzw. bis zum Erreichen\nder Fachhochschulreife hervorgeht oder\nII. ein Notenspiegel, aus dem die bestandene Pr\u00fcfungsleistung \u00fcber eine\nLehrveranstaltung im Rahmen des grundst\u00e4ndigen Studiums hervorgeht, die die\nenglische Sprache zum Inhalt hatte oder\nIII. eine Bescheinigung \u00fcber den mindestens sechsmonatigen Aufenthalt an einer Schule, Hochschule oder anderen Bildungsinstitution mit Englisch als\nUnterrichtssprache oder\nIV. eine Bescheinigung \u00fcber den Aufenthalt im englischsprachigen Ausland, der einen Zeitraum von mindestens sechs Monaten bzw. einem Studiensemester umfasst.\nDie Vorlage anderer geeigneter Nachweise ist m\u00f6glich.\n(2) Auswahlkriterien nach \u00a7 9 Abs. 2\n1. Ergebnis eines Auswahlgespr\u00e4chs\nNicht zutreffend.\n2. Leistungen, die mit der Abschlusspr\u00fcfung des grundst\u00e4ndigen Studiums nach Abs. 1\ni. V. m. \u00a7 5 Abs. 1 Nr. 1 nachgewiesen sind\nDie Durchschnittsnote der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums nach\nAbs. 1 bildet die Teilnote 1 als Basis zur Bestimmung der Auswahlnote.  Abweichend von Satz\n1 bildet in den F\u00e4llen des \u00a7 3 Abs. 2 Nr. 1 Satz 2 die Durchschnittsnote nach \u00a7 3 Abs. 2 Nr. 1 Satz 3 die Teilnote 1. Bei ausl\u00e4ndischen Bildungsnachweisen ist die Durchschnittsnote nach\ndeutsc her Deutung als Teilnote 1 zu ber\u00fccksichtigen.\nZus\u00e4tzlich werden die Einzelnoten folgender F\u00e4cher der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums, die \u00fcber die Eignung f\u00fcr den gew\u00e4hlten Studiengang\nbesonderen Aufschluss geben, f\u00fcr die Auswahl herangezogen:\n- Technische Mechanik (Dynamik),\n- Elektrotechnik,\n- Messtechnik,\n- Regelungstechnik,\n- Elektrische Antriebe.\nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. F\u00e4cher jeweils mit dem Wert 0,1\nbewertet. Die kumulierte Gesamtzahl bildet die Teil note 2.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 21a Mechatronik (MME) Vollzeitstudium\n(1) Studiengangspezifische Zugangsvoraussetzungen gem\u00e4\u00df \u00a7 5  Abs. 1\nZugangsvoraussetzungen f\u00fcr den Masterstudiengang Mechatronik sind:\n1. Ein mit der Note 2,9 oder besser abgeschlossenes grundst\u00e4ndiges Hochschulstudium\ngem\u00e4\u00df \u00a7 5 Abs. 1 Nr. 1 in einem Studiengang der Fachrichtungen Maschinenbau,\nElektrotechnik, Fahrzeugtechnik, Mechatronik, Feinwerktechnik oder einer verwandten\nFachrichtung.\n2. Englischkenntnisse, \u00e4quivalent zu Niveau- Stufe B1 des Europ\u00e4ischen Referenzrahmens\nf\u00fcr das Lernen und Lehren von Fremdsprachen. Als \u00e4quivalent zu einem Zertifikat \u00fcber die\nNiveau -Stufe B1 gelten insbesondere folgende Nachweise:\nI. das Schulabschlusszeugnis, aus dem der Besuch des Englischunterrichts bis zum\nErreichen des mittleren Bildungsabschlusses (10. Klass e) bzw. bis zum Erreichen\nder Fachhochschulreife hervorgeht oder\nII. ein Notenspiegel, aus dem die bestandene Pr\u00fcfungsleistung \u00fcber eine\nLehrveranstaltung im Rahmen des grundst\u00e4ndigen Studiums hervorgeht, die die\nenglische Sprache zum Inhalt hatte oder\nIII. eine Bescheinigung \u00fcber den mindestens sechsmonatigen Aufenthalt an einer Schule, Hochschule oder anderen Bildungsinstitution mit Englisch als\nUnterrichtssprache oder\nIV. eine Bescheinigung \u00fcber den Aufenthalt im englischsprachigen Ausland, der einen Zeitraum von mindestens sechs Monaten bzw. einem Studiensemester umfasst.\nDie Vorlage anderer geeigneter Nachweise ist m\u00f6glich.\n(2) Auswahlkriterien nach \u00a7 9 Abs. 2\n1. Ergebnis eines Auswahlgespr\u00e4chs\nNicht zutreffend.\n2. Leistungen, die mit der Abschlusspr\u00fcfung des grundst\u00e4ndigen Studiums nach Abs. 1\ni. V. m. \u00a7 5 Abs. 1 Nr. 1 nachgewiesen sind\nDie Durchschnittsnote der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums nach\nAbs. 1 bildet die Teilnote 1 als Basis zur Bestimmung der Auswahlnote. Abweichend von Satz\n1 bildet in den F\u00e4llen des \u00a7 3 Abs. 2 Nr. 1 Satz 2 die Durchschnittsnote nach \u00a7 3 Abs. 2 Nr. 1\nSatz 3 die Teilnote 1. Bei ausl\u00e4ndischen Bildungsnachweisen ist die Durchschnittsnote nach\ndeutscher Deutung als Teilnote 1 zu ber\u00fccksichtigen.\nZus\u00e4tzlich werden die Einzelnoten folgender F\u00e4cher der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums, die \u00fcber die Eignung f\u00fcr den gew\u00e4hlten Studiengang\nbesonderen Aufschluss geben, f\u00fcr die Auswahl herangezogen:\n- Technische Mechanik (Dynamik),\n- Elektrotechnik,\n- Messt echnik,\n- Regelungstechnik,\n- Elektrische Antriebe.\nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. F\u00e4cher jeweils mit dem Wert 0,1\nbewertet. Die kumulierte Gesamtzahl bildet die Teilnote 2.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 9 Zugangs - und Auswahlkriterien in den Masterstudieng\u00e4ngen\n(1)  1Im Besonderen Teil (\u00a7\u00a7 12 -26) dieser Satzung k\u00f6nnen ein oder mehrere der in Absatz 2\ngenannten Auswahlkriterien als weitere Zugangskriterien festgelegt werden. 2N\u00e4heres\nregelt der B esondere Teil f\u00fcr den jeweiligen Studiengang (\u00a7\u00a7 12 -26).\n(2)  1F\u00fcr die Bildung der Ranglisten f\u00fcr das erste Fachsemester in den Masterstudieng\u00e4ngen\nwird, neben dem Ergebnis des fachlich einschl\u00e4gigen Hochschulabschlusses oder des\ngleichwertigen Abschlusses,  mindestens eines der folgenden  Auswahlkriterien\nber\u00fccksichtigt:\n1. Leistungen, die in dem Studium erbracht wurden, das Voraussetzung f\u00fcr den Zugang\nzu dem Masterstudiengang ist ,   Seite 8 von 43 2. Englischkenntnisse , n\u00e4heres regelt der Besondere Teil f\u00fcr den jeweiligen Studiengang\n(\u00a7\u00a7 12 -26),\n3. Berufst\u00e4tigkeit und Qualifikationen:\na) Art einer abgeschlossenen Berufsausbildung oder einer Berufst\u00e4tigkeit in einem\nanerkannten Ausbildungsberuf  oder eine andere einschl\u00e4gige Berufst\u00e4tigkeit , die \u00fcber\ndie fachspezifische Eignung Auskunft gibt, jeweils  einzeln und in Kombination, und\nb) Qualifikation en, die \u00fcber die fachspezifische Leistung Auskunft geben, jeweils einzeln\noder in Kombination,\n4. das Ergebnis eines fachspezifischen Studieneignungstests ,\n5. das Ergebnis des Auswahlgespr\u00e4chs/anderen m\u00fcndlichen Verfahrens  gem\u00e4\u00df \u00a7 9a ,\n6. ein Motivationsschreiben,\n7. eine schriftliche Abhandlung (Essay).\n2N\u00e4heres sowie die Gewichtung regelt der B esondere Teil f\u00fcr den jeweiligen Studiengang (\u00a7\u00a7\n12-26).\n(2) 1Die Auswahl f\u00fcr h\u00f6here Fachsemester erfolgt gem\u00e4\u00df \u00a7 7 HZG i. V. m. \u00a7 32 HZVO.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}]
context = [entry['page_content'] for entry in data]
eval_prompt = create_eval_prompt("""Welche Dokumente kÃ¶nnen als Nachweis fÃ¼r deutsche Sprachkenntnisse akzeptiert werden?""", str(context))
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

awq_model.eval()
with torch.no_grad():
    with torch.cuda.amp.autocast():
        print(tokenizer.decode(awq_model.generate(**model_input, max_new_tokens=500, repetition_penalty=1.15)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


AssertionError: AWQ kernels could not be loaded. Please install them from https://github.com/casper-hansen/AutoAWQ_kernels

## test model

In [3]:
def create_eval_prompt(query: str, context: str):
    system_prompt = "You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. It is mandatory to answer in GERMAN:\n\n"
    return f"[INST]{system_prompt}Context: {context}\n\nQuestion: {query}[/INST]"

In [5]:
import torch

data = [{"page_content": "\n\u00a7 4 Sprachkenntnisse\n(1) 1Neben den allgemeinen Zugangsvoraussetzungen (\u00a7 59 LHG) sind f\u00fcr die in \u00a7 1 Abs. 1\nS. 1 genannten Studieng\u00e4nge deutsche Sprach kenntnisse nachzuweisen. 2Diese k\u00f6nnen\ndurch eine deutsche Hochschulzugangsberechtigung (u. a. erfolgreich abgeschlossenes\ngrundst\u00e4ndiges Hochschulstudium) nachgewiesen werden. 3Ferner kann der\nSprachnachweis durch die Vorlage eines der folgenden Dokumente erbracht werden:\n1. Feststellungspr\u00fcfung f\u00fcr ein Bachelorstudium durch Vorlage der Zugangsberechtigung\ndes Studienkollegs an der Hochschule Konstanz,\n2. Test Deutsch als Fremdsprache (TestDaF), sofern im Durchschnitt mindestens die\nStufe TDN 4 erreicht wurde,   Seite 5 von 43 3. Deutsche Sprachpr\u00fcfung f\u00fcr den Hochschulzugang (DSH), sofern die DSH mit\nmindestens der Stufe DSH -2 abgeschlossen wurde,\n4. \u201eTelc Deutsch C1 Hochschule\u201c\noder eine \u00e4quivalente Sprachpr\u00fcfung gem\u00e4\u00df der Rahmenordnung \u00fcber Deutsche\nSprachpr\u00fcfungen f\u00fcr das Studium an deutschen Hochschulen (RO -DT). 4Auf den Nachweis\neiner deutschen Sprachpr\u00fcfung kann bei Bewerber innen und Bewerbern im besonders\nbegr\u00fcndeten Einzelfall verzichtet werden, insbesondere wenn sie die deutsche\nStaatsangeh\u00f6rigkeit besitzen.\n(2) 1Sprachnachweise f\u00fcr den gew\u00e4hl ten Studiengang, die durch die Bewerberin oder den\nBewerber bis zum Bewerbungsschluss nicht vorgelegt werden k\u00f6nnen, k\u00f6nnen bis zum\nVorlesungsbeginn des Semesters gem\u00e4\u00df Terminplan der Hochschule Konstanz, f\u00fcr das  der\nAntrag auf Zulassung gestellt wurde, nachgereicht werden. 2Die Zulassung erfolgt in diesem\nFall gem \u00e4\u00df \u00a7 6  Abs. 5  unter Vorbehalt .\n(3) 1F\u00fcr Zeitstudierende gelten die Regelungen in \u00a7 10 Zulassungs - und\nImmatrikulationsordnung (ZIO) der Hochschule Konstanz.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 21b Mechatronik (MME) Berufsbegleitendes Studium\n(1) Studiengangspezifische Zugangsvoraussetzungen gem\u00e4\u00df \u00a7 5  Abs. 1\nZugangsvoraussetzungen f\u00fcr den Masterstudiengang Mechatronik sind:\n1. Ein mit der Note 2,9 oder besser abgeschlossenes grundst\u00e4ndiges Hochschulstudium\ngem\u00e4\u00df \u00a7 5 Abs. 1 Nr. 1 in einem Studiengang der Fachrichtungen Systemtechnik,\nMaschinenbau, Elektrotechnik, Fahrzeugtechnik, Mechatronik, Feinwerktechnik oder einer\nverwandten Fachrichtung.\n2. Englischkenntnisse, \u00e4quivalent z u Niveau- Stufe B1 des Europ\u00e4ischen Referenzrahmens\nf\u00fcr das Lernen und Lehren von Fremdsprachen. Als \u00e4quivalent zu einem Zertifikat \u00fcber die\nNiveau -Stufe B1 gelten insbesondere folgende Nachweise:\nI. das Schulabschlusszeugnis, aus dem der Besuch des Englischunterrichts bis zum\nErreichen des mittleren Bildungsabschlusses (10. Klasse) bzw. bis zum Erreichen\nder Fachhochschulreife hervorgeht oder\nII. ein Notenspiegel, aus dem die bestandene Pr\u00fcfungsleistung \u00fcber eine\nLehrveranstaltung im Rahmen des grundst\u00e4ndigen Studiums hervorgeht, die die\nenglische Sprache zum Inhalt hatte oder\nIII. eine Bescheinigung \u00fcber den mindestens sechsmonatigen Aufenthalt an einer Schule, Hochschule oder anderen Bildungsinstitution mit Englisch als\nUnterrichtssprache oder\nIV. eine Bescheinigung \u00fcber den Aufenthalt im englischsprachigen Ausland, der einen Zeitraum von mindestens sechs Monaten bzw. einem Studiensemester umfasst.\nDie Vorlage anderer geeigneter Nachweise ist m\u00f6glich.\n(2) Auswahlkriterien nach \u00a7 9 Abs. 2\n1. Ergebnis eines Auswahlgespr\u00e4chs\nNicht zutreffend.\n2. Leistungen, die mit der Abschlusspr\u00fcfung des grundst\u00e4ndigen Studiums nach Abs. 1\ni. V. m. \u00a7 5 Abs. 1 Nr. 1 nachgewiesen sind\nDie Durchschnittsnote der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums nach\nAbs. 1 bildet die Teilnote 1 als Basis zur Bestimmung der Auswahlnote.  Abweichend von Satz\n1 bildet in den F\u00e4llen des \u00a7 3 Abs. 2 Nr. 1 Satz 2 die Durchschnittsnote nach \u00a7 3 Abs. 2 Nr. 1 Satz 3 die Teilnote 1. Bei ausl\u00e4ndischen Bildungsnachweisen ist die Durchschnittsnote nach\ndeutsc her Deutung als Teilnote 1 zu ber\u00fccksichtigen.\nZus\u00e4tzlich werden die Einzelnoten folgender F\u00e4cher der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums, die \u00fcber die Eignung f\u00fcr den gew\u00e4hlten Studiengang\nbesonderen Aufschluss geben, f\u00fcr die Auswahl herangezogen:\n- Technische Mechanik (Dynamik),\n- Elektrotechnik,\n- Messtechnik,\n- Regelungstechnik,\n- Elektrische Antriebe.\nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. F\u00e4cher jeweils mit dem Wert 0,1\nbewertet. Die kumulierte Gesamtzahl bildet die Teil note 2.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 21a Mechatronik (MME) Vollzeitstudium\n(1) Studiengangspezifische Zugangsvoraussetzungen gem\u00e4\u00df \u00a7 5  Abs. 1\nZugangsvoraussetzungen f\u00fcr den Masterstudiengang Mechatronik sind:\n1. Ein mit der Note 2,9 oder besser abgeschlossenes grundst\u00e4ndiges Hochschulstudium\ngem\u00e4\u00df \u00a7 5 Abs. 1 Nr. 1 in einem Studiengang der Fachrichtungen Maschinenbau,\nElektrotechnik, Fahrzeugtechnik, Mechatronik, Feinwerktechnik oder einer verwandten\nFachrichtung.\n2. Englischkenntnisse, \u00e4quivalent zu Niveau- Stufe B1 des Europ\u00e4ischen Referenzrahmens\nf\u00fcr das Lernen und Lehren von Fremdsprachen. Als \u00e4quivalent zu einem Zertifikat \u00fcber die\nNiveau -Stufe B1 gelten insbesondere folgende Nachweise:\nI. das Schulabschlusszeugnis, aus dem der Besuch des Englischunterrichts bis zum\nErreichen des mittleren Bildungsabschlusses (10. Klass e) bzw. bis zum Erreichen\nder Fachhochschulreife hervorgeht oder\nII. ein Notenspiegel, aus dem die bestandene Pr\u00fcfungsleistung \u00fcber eine\nLehrveranstaltung im Rahmen des grundst\u00e4ndigen Studiums hervorgeht, die die\nenglische Sprache zum Inhalt hatte oder\nIII. eine Bescheinigung \u00fcber den mindestens sechsmonatigen Aufenthalt an einer Schule, Hochschule oder anderen Bildungsinstitution mit Englisch als\nUnterrichtssprache oder\nIV. eine Bescheinigung \u00fcber den Aufenthalt im englischsprachigen Ausland, der einen Zeitraum von mindestens sechs Monaten bzw. einem Studiensemester umfasst.\nDie Vorlage anderer geeigneter Nachweise ist m\u00f6glich.\n(2) Auswahlkriterien nach \u00a7 9 Abs. 2\n1. Ergebnis eines Auswahlgespr\u00e4chs\nNicht zutreffend.\n2. Leistungen, die mit der Abschlusspr\u00fcfung des grundst\u00e4ndigen Studiums nach Abs. 1\ni. V. m. \u00a7 5 Abs. 1 Nr. 1 nachgewiesen sind\nDie Durchschnittsnote der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums nach\nAbs. 1 bildet die Teilnote 1 als Basis zur Bestimmung der Auswahlnote. Abweichend von Satz\n1 bildet in den F\u00e4llen des \u00a7 3 Abs. 2 Nr. 1 Satz 2 die Durchschnittsnote nach \u00a7 3 Abs. 2 Nr. 1\nSatz 3 die Teilnote 1. Bei ausl\u00e4ndischen Bildungsnachweisen ist die Durchschnittsnote nach\ndeutscher Deutung als Teilnote 1 zu ber\u00fccksichtigen.\nZus\u00e4tzlich werden die Einzelnoten folgender F\u00e4cher der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums, die \u00fcber die Eignung f\u00fcr den gew\u00e4hlten Studiengang\nbesonderen Aufschluss geben, f\u00fcr die Auswahl herangezogen:\n- Technische Mechanik (Dynamik),\n- Elektrotechnik,\n- Messt echnik,\n- Regelungstechnik,\n- Elektrische Antriebe.\nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. F\u00e4cher jeweils mit dem Wert 0,1\nbewertet. Die kumulierte Gesamtzahl bildet die Teilnote 2.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 9 Zugangs - und Auswahlkriterien in den Masterstudieng\u00e4ngen\n(1)  1Im Besonderen Teil (\u00a7\u00a7 12 -26) dieser Satzung k\u00f6nnen ein oder mehrere der in Absatz 2\ngenannten Auswahlkriterien als weitere Zugangskriterien festgelegt werden. 2N\u00e4heres\nregelt der B esondere Teil f\u00fcr den jeweiligen Studiengang (\u00a7\u00a7 12 -26).\n(2)  1F\u00fcr die Bildung der Ranglisten f\u00fcr das erste Fachsemester in den Masterstudieng\u00e4ngen\nwird, neben dem Ergebnis des fachlich einschl\u00e4gigen Hochschulabschlusses oder des\ngleichwertigen Abschlusses,  mindestens eines der folgenden  Auswahlkriterien\nber\u00fccksichtigt:\n1. Leistungen, die in dem Studium erbracht wurden, das Voraussetzung f\u00fcr den Zugang\nzu dem Masterstudiengang ist ,   Seite 8 von 43 2. Englischkenntnisse , n\u00e4heres regelt der Besondere Teil f\u00fcr den jeweiligen Studiengang\n(\u00a7\u00a7 12 -26),\n3. Berufst\u00e4tigkeit und Qualifikationen:\na) Art einer abgeschlossenen Berufsausbildung oder einer Berufst\u00e4tigkeit in einem\nanerkannten Ausbildungsberuf  oder eine andere einschl\u00e4gige Berufst\u00e4tigkeit , die \u00fcber\ndie fachspezifische Eignung Auskunft gibt, jeweils  einzeln und in Kombination, und\nb) Qualifikation en, die \u00fcber die fachspezifische Leistung Auskunft geben, jeweils einzeln\noder in Kombination,\n4. das Ergebnis eines fachspezifischen Studieneignungstests ,\n5. das Ergebnis des Auswahlgespr\u00e4chs/anderen m\u00fcndlichen Verfahrens  gem\u00e4\u00df \u00a7 9a ,\n6. ein Motivationsschreiben,\n7. eine schriftliche Abhandlung (Essay).\n2N\u00e4heres sowie die Gewichtung regelt der B esondere Teil f\u00fcr den jeweiligen Studiengang (\u00a7\u00a7\n12-26).\n(2) 1Die Auswahl f\u00fcr h\u00f6here Fachsemester erfolgt gem\u00e4\u00df \u00a7 7 HZG i. V. m. \u00a7 32 HZVO.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}]
context = [entry['page_content'] for entry in data]
eval_prompt = create_eval_prompt("""Welche Dokumente kÃ¶nnen als Nachweis fÃ¼r deutsche Sprachkenntnisse akzeptiert werden?""", str(context))
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cpu")

model.eval()
with torch.no_grad():
    with torch.cuda.amp.autocast():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=500, repetition_penalty=1.15)[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyError: 'Cache only has 0 layers, attempted to access layer with index 0'

In [ ]:
!curl -L -o convert.py https://github.com/ggerganov/llama.cpp/raw/master/convert.py
!curl -L -o requirements.txt https://github.com/ggerganov/llama.cpp/raw/master/requirements.txt

!pip install -r requirements.txt

# Convert the 7B model to ggml FP16 format
!python3 convert.py model

## Code below is optional - uncomment (remove leftmost '# ') to use

# # [Optional] for models using BPE tokenizers
# !python convert.py model --vocabtype bpe

# # [Optional] quantize the model to 4-bits (using q4_0 method)
# !./quantize ./model/ggml-model-f16.gguf ./model/ggml-model-q4_0.gguf q4_0

# # [Optional] update the gguf filetype to current if older version is unsupported by another application
# !./quantize ./model/ggml-model-q4_0.gguf ./model/ggml-model-q4_0-v2.gguf COPY

# VIA UNSLOTH


- unsloth provides less ressource intensive and faster finetuning + conversion
- see https://github.com/unslothai/unsloth for installation instructions

In [4]:
import torch
torch.cuda.is_bf16_supported()
torch.cuda.get_device_capability()

(7, 5)

In [4]:
from unsloth import FastLanguageModel
import torch

#"checkpoints/raft-completion-only/checkpoint-350"
model, tokenizer = FastLanguageModel.from_pretrained("checkpoints/raft-v2/checkpoint-250", load_in_4bit = True, dtype=torch.bfloat16)
#model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method = "q4_k_m")

==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.0.dev0.
   \\   /|    GPU: NVIDIA TITAN RTX. Max memory: 23.643 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Device does not support bfloat16. Will change to float16.
Unsloth: Will load checkpoints/raft-v2/checkpoint-250 as a legacy tokenizer.


In [2]:
tokenizer

LlamaTokenizer(name_or_path='checkpoints/raft-v2/checkpoint-250', vocab_size=32768, model_max_length=32768, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[/INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[TOOL_CALLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("[AVAILABLE_TOOLS]", 

In [5]:
model.config.to_dict()["torch_dtype"]

'float16'

In [ ]:
'''
# https://github.com/ggerganov/llama.cpp/blob/master/examples/quantize/quantize.cpp#L19
# From https://mlabonne.github.io/blog/posts/Quantize_Llama_2_models_using_ggml.html
ALLOWED_QUANTS = \
{
    "not_quantized"  : "Recommended. Fast conversion. Slow inference, big files.",
    "fast_quantized" : "Recommended. Fast conversion. OK inference, OK file size.",
    "quantized"      : "Recommended. Slow conversion. Fast inference, small files.",
    "f32"     : "Not recommended. Retains 100% accuracy, but super slow and memory hungry.",
    "f16"     : "Fastest conversion + retains 100% accuracy. Slow and memory hungry.",
    "q8_0"    : "Fast conversion. High resource use, but generally acceptable.",
    "q4_k_m"  : "Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K",
    "q5_k_m"  : "Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K",
    "q2_k"    : "Uses Q4_K for the attention.vw and feed_forward.w2 tensors, Q2_K for the other tensors.",
    "q3_k_l"  : "Uses Q5_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K",
    "q3_k_m"  : "Uses Q4_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K",
    "q3_k_s"  : "Uses Q3_K for all tensors",
    "q4_0"    : "Original quant method, 4-bit.",
    "q4_1"    : "Higher accuracy than q4_0 but not as high as q5_0. However has quicker inference than q5 models.",
    "q4_k_s"  : "Uses Q4_K for all tensors",
    "q4_k"    : "alias for q4_k_m",
    "q5_k"    : "alias for q5_k_m",
    "q5_0"    : "Higher accuracy, higher resource usage and slower inference.",
    "q5_1"    : "Even higher accuracy, resource usage and slower inference.",
    "q5_k_s"  : "Uses Q5_K for all tensors",
    "q6_k"    : "Uses Q8_K for all tensors",
    "iq2_xxs" : "2.06 bpw quantization",
    "iq2_xs"  : "2.31 bpw quantization",
    "iq3_xxs" : "3.06 bpw quantization",
    "q3_k_xs" : "3-bit extra small quantization",
}
'''

In [ ]:
# if save_pretrained_gguf fails

#model.save_pretrained_merged("model/mistral-rag-instruct", tokenizer, save_method="merged_16bit")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 178.35 out of 251.41 RAM for saving.


100%|██████████| 32/32 [00:01<00:00, 31.18it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


In [2]:
#model.save_pretrained_gguf("model/mistral-rag-instruct", tokenizer, quantization_method = "q8_0") # Save to 8bit Q8_0
model.save_pretrained_gguf("model/mistral-rag-instruct", tokenizer, quantization_method = "q4_k_m") # Save to 4bit q4_k_m

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 202.92 out of 251.39 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 35.59it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting mistral model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model/mistral-rag-instruct into f16 GGUF format.
The output location will be ./model/mistral-rag-instruct/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: mistral-rag-instruct
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00003.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {4096, 32768}
INFO:

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig



base_model_id = "mistralai/Mistral-7B-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id) #, add_bos_token=True, trust_remote_code=True

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "checkpoints/raft-v2/checkpoint-250")

In [8]:
def create_eval_prompt(query: str, context: str):
    system_prompt = "You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. It is mandatory to answer in GERMAN:\n\n"
    return f"[INST]{system_prompt}Context: {context}\n\nQuestion: {query}[/INST]"

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
ft_model = AutoModelForCausalLM.from_pretrained(
    "merged_model", local_files_only=True, device_map='auto'
)
eval_tokenizer = AutoTokenizer.from_pretrained("merged_model")

OSError: merged_model is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
data = [{"page_content": "\n\u00a7 4 Sprachkenntnisse\n(1) 1Neben den allgemeinen Zugangsvoraussetzungen (\u00a7 59 LHG) sind f\u00fcr die in \u00a7 1 Abs. 1\nS. 1 genannten Studieng\u00e4nge deutsche Sprach kenntnisse nachzuweisen. 2Diese k\u00f6nnen\ndurch eine deutsche Hochschulzugangsberechtigung (u. a. erfolgreich abgeschlossenes\ngrundst\u00e4ndiges Hochschulstudium) nachgewiesen werden. 3Ferner kann der\nSprachnachweis durch die Vorlage eines der folgenden Dokumente erbracht werden:\n1. Feststellungspr\u00fcfung f\u00fcr ein Bachelorstudium durch Vorlage der Zugangsberechtigung\ndes Studienkollegs an der Hochschule Konstanz,\n2. Test Deutsch als Fremdsprache (TestDaF), sofern im Durchschnitt mindestens die\nStufe TDN 4 erreicht wurde,   Seite 5 von 43 3. Deutsche Sprachpr\u00fcfung f\u00fcr den Hochschulzugang (DSH), sofern die DSH mit\nmindestens der Stufe DSH -2 abgeschlossen wurde,\n4. \u201eTelc Deutsch C1 Hochschule\u201c\noder eine \u00e4quivalente Sprachpr\u00fcfung gem\u00e4\u00df der Rahmenordnung \u00fcber Deutsche\nSprachpr\u00fcfungen f\u00fcr das Studium an deutschen Hochschulen (RO -DT). 4Auf den Nachweis\neiner deutschen Sprachpr\u00fcfung kann bei Bewerber innen und Bewerbern im besonders\nbegr\u00fcndeten Einzelfall verzichtet werden, insbesondere wenn sie die deutsche\nStaatsangeh\u00f6rigkeit besitzen.\n(2) 1Sprachnachweise f\u00fcr den gew\u00e4hl ten Studiengang, die durch die Bewerberin oder den\nBewerber bis zum Bewerbungsschluss nicht vorgelegt werden k\u00f6nnen, k\u00f6nnen bis zum\nVorlesungsbeginn des Semesters gem\u00e4\u00df Terminplan der Hochschule Konstanz, f\u00fcr das  der\nAntrag auf Zulassung gestellt wurde, nachgereicht werden. 2Die Zulassung erfolgt in diesem\nFall gem \u00e4\u00df \u00a7 6  Abs. 5  unter Vorbehalt .\n(3) 1F\u00fcr Zeitstudierende gelten die Regelungen in \u00a7 10 Zulassungs - und\nImmatrikulationsordnung (ZIO) der Hochschule Konstanz.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 21b Mechatronik (MME) Berufsbegleitendes Studium\n(1) Studiengangspezifische Zugangsvoraussetzungen gem\u00e4\u00df \u00a7 5  Abs. 1\nZugangsvoraussetzungen f\u00fcr den Masterstudiengang Mechatronik sind:\n1. Ein mit der Note 2,9 oder besser abgeschlossenes grundst\u00e4ndiges Hochschulstudium\ngem\u00e4\u00df \u00a7 5 Abs. 1 Nr. 1 in einem Studiengang der Fachrichtungen Systemtechnik,\nMaschinenbau, Elektrotechnik, Fahrzeugtechnik, Mechatronik, Feinwerktechnik oder einer\nverwandten Fachrichtung.\n2. Englischkenntnisse, \u00e4quivalent z u Niveau- Stufe B1 des Europ\u00e4ischen Referenzrahmens\nf\u00fcr das Lernen und Lehren von Fremdsprachen. Als \u00e4quivalent zu einem Zertifikat \u00fcber die\nNiveau -Stufe B1 gelten insbesondere folgende Nachweise:\nI. das Schulabschlusszeugnis, aus dem der Besuch des Englischunterrichts bis zum\nErreichen des mittleren Bildungsabschlusses (10. Klasse) bzw. bis zum Erreichen\nder Fachhochschulreife hervorgeht oder\nII. ein Notenspiegel, aus dem die bestandene Pr\u00fcfungsleistung \u00fcber eine\nLehrveranstaltung im Rahmen des grundst\u00e4ndigen Studiums hervorgeht, die die\nenglische Sprache zum Inhalt hatte oder\nIII. eine Bescheinigung \u00fcber den mindestens sechsmonatigen Aufenthalt an einer Schule, Hochschule oder anderen Bildungsinstitution mit Englisch als\nUnterrichtssprache oder\nIV. eine Bescheinigung \u00fcber den Aufenthalt im englischsprachigen Ausland, der einen Zeitraum von mindestens sechs Monaten bzw. einem Studiensemester umfasst.\nDie Vorlage anderer geeigneter Nachweise ist m\u00f6glich.\n(2) Auswahlkriterien nach \u00a7 9 Abs. 2\n1. Ergebnis eines Auswahlgespr\u00e4chs\nNicht zutreffend.\n2. Leistungen, die mit der Abschlusspr\u00fcfung des grundst\u00e4ndigen Studiums nach Abs. 1\ni. V. m. \u00a7 5 Abs. 1 Nr. 1 nachgewiesen sind\nDie Durchschnittsnote der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums nach\nAbs. 1 bildet die Teilnote 1 als Basis zur Bestimmung der Auswahlnote.  Abweichend von Satz\n1 bildet in den F\u00e4llen des \u00a7 3 Abs. 2 Nr. 1 Satz 2 die Durchschnittsnote nach \u00a7 3 Abs. 2 Nr. 1 Satz 3 die Teilnote 1. Bei ausl\u00e4ndischen Bildungsnachweisen ist die Durchschnittsnote nach\ndeutsc her Deutung als Teilnote 1 zu ber\u00fccksichtigen.\nZus\u00e4tzlich werden die Einzelnoten folgender F\u00e4cher der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums, die \u00fcber die Eignung f\u00fcr den gew\u00e4hlten Studiengang\nbesonderen Aufschluss geben, f\u00fcr die Auswahl herangezogen:\n- Technische Mechanik (Dynamik),\n- Elektrotechnik,\n- Messtechnik,\n- Regelungstechnik,\n- Elektrische Antriebe.\nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. F\u00e4cher jeweils mit dem Wert 0,1\nbewertet. Die kumulierte Gesamtzahl bildet die Teil note 2.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 21a Mechatronik (MME) Vollzeitstudium\n(1) Studiengangspezifische Zugangsvoraussetzungen gem\u00e4\u00df \u00a7 5  Abs. 1\nZugangsvoraussetzungen f\u00fcr den Masterstudiengang Mechatronik sind:\n1. Ein mit der Note 2,9 oder besser abgeschlossenes grundst\u00e4ndiges Hochschulstudium\ngem\u00e4\u00df \u00a7 5 Abs. 1 Nr. 1 in einem Studiengang der Fachrichtungen Maschinenbau,\nElektrotechnik, Fahrzeugtechnik, Mechatronik, Feinwerktechnik oder einer verwandten\nFachrichtung.\n2. Englischkenntnisse, \u00e4quivalent zu Niveau- Stufe B1 des Europ\u00e4ischen Referenzrahmens\nf\u00fcr das Lernen und Lehren von Fremdsprachen. Als \u00e4quivalent zu einem Zertifikat \u00fcber die\nNiveau -Stufe B1 gelten insbesondere folgende Nachweise:\nI. das Schulabschlusszeugnis, aus dem der Besuch des Englischunterrichts bis zum\nErreichen des mittleren Bildungsabschlusses (10. Klass e) bzw. bis zum Erreichen\nder Fachhochschulreife hervorgeht oder\nII. ein Notenspiegel, aus dem die bestandene Pr\u00fcfungsleistung \u00fcber eine\nLehrveranstaltung im Rahmen des grundst\u00e4ndigen Studiums hervorgeht, die die\nenglische Sprache zum Inhalt hatte oder\nIII. eine Bescheinigung \u00fcber den mindestens sechsmonatigen Aufenthalt an einer Schule, Hochschule oder anderen Bildungsinstitution mit Englisch als\nUnterrichtssprache oder\nIV. eine Bescheinigung \u00fcber den Aufenthalt im englischsprachigen Ausland, der einen Zeitraum von mindestens sechs Monaten bzw. einem Studiensemester umfasst.\nDie Vorlage anderer geeigneter Nachweise ist m\u00f6glich.\n(2) Auswahlkriterien nach \u00a7 9 Abs. 2\n1. Ergebnis eines Auswahlgespr\u00e4chs\nNicht zutreffend.\n2. Leistungen, die mit der Abschlusspr\u00fcfung des grundst\u00e4ndigen Studiums nach Abs. 1\ni. V. m. \u00a7 5 Abs. 1 Nr. 1 nachgewiesen sind\nDie Durchschnittsnote der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums nach\nAbs. 1 bildet die Teilnote 1 als Basis zur Bestimmung der Auswahlnote. Abweichend von Satz\n1 bildet in den F\u00e4llen des \u00a7 3 Abs. 2 Nr. 1 Satz 2 die Durchschnittsnote nach \u00a7 3 Abs. 2 Nr. 1\nSatz 3 die Teilnote 1. Bei ausl\u00e4ndischen Bildungsnachweisen ist die Durchschnittsnote nach\ndeutscher Deutung als Teilnote 1 zu ber\u00fccksichtigen.\nZus\u00e4tzlich werden die Einzelnoten folgender F\u00e4cher der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums, die \u00fcber die Eignung f\u00fcr den gew\u00e4hlten Studiengang\nbesonderen Aufschluss geben, f\u00fcr die Auswahl herangezogen:\n- Technische Mechanik (Dynamik),\n- Elektrotechnik,\n- Messt echnik,\n- Regelungstechnik,\n- Elektrische Antriebe.\nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. F\u00e4cher jeweils mit dem Wert 0,1\nbewertet. Die kumulierte Gesamtzahl bildet die Teilnote 2.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 9 Zugangs - und Auswahlkriterien in den Masterstudieng\u00e4ngen\n(1)  1Im Besonderen Teil (\u00a7\u00a7 12 -26) dieser Satzung k\u00f6nnen ein oder mehrere der in Absatz 2\ngenannten Auswahlkriterien als weitere Zugangskriterien festgelegt werden. 2N\u00e4heres\nregelt der B esondere Teil f\u00fcr den jeweiligen Studiengang (\u00a7\u00a7 12 -26).\n(2)  1F\u00fcr die Bildung der Ranglisten f\u00fcr das erste Fachsemester in den Masterstudieng\u00e4ngen\nwird, neben dem Ergebnis des fachlich einschl\u00e4gigen Hochschulabschlusses oder des\ngleichwertigen Abschlusses,  mindestens eines der folgenden  Auswahlkriterien\nber\u00fccksichtigt:\n1. Leistungen, die in dem Studium erbracht wurden, das Voraussetzung f\u00fcr den Zugang\nzu dem Masterstudiengang ist ,   Seite 8 von 43 2. Englischkenntnisse , n\u00e4heres regelt der Besondere Teil f\u00fcr den jeweiligen Studiengang\n(\u00a7\u00a7 12 -26),\n3. Berufst\u00e4tigkeit und Qualifikationen:\na) Art einer abgeschlossenen Berufsausbildung oder einer Berufst\u00e4tigkeit in einem\nanerkannten Ausbildungsberuf  oder eine andere einschl\u00e4gige Berufst\u00e4tigkeit , die \u00fcber\ndie fachspezifische Eignung Auskunft gibt, jeweils  einzeln und in Kombination, und\nb) Qualifikation en, die \u00fcber die fachspezifische Leistung Auskunft geben, jeweils einzeln\noder in Kombination,\n4. das Ergebnis eines fachspezifischen Studieneignungstests ,\n5. das Ergebnis des Auswahlgespr\u00e4chs/anderen m\u00fcndlichen Verfahrens  gem\u00e4\u00df \u00a7 9a ,\n6. ein Motivationsschreiben,\n7. eine schriftliche Abhandlung (Essay).\n2N\u00e4heres sowie die Gewichtung regelt der B esondere Teil f\u00fcr den jeweiligen Studiengang (\u00a7\u00a7\n12-26).\n(2) 1Die Auswahl f\u00fcr h\u00f6here Fachsemester erfolgt gem\u00e4\u00df \u00a7 7 HZG i. V. m. \u00a7 32 HZVO.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}]
context = [entry['page_content'] for entry in data]
eval_prompt = create_eval_prompt("""Welche Dokumente kÃ¶nnen als Nachweis fÃ¼r deutsche Sprachkenntnisse akzeptiert werden?""", str(context))
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=500, repetition_penalty=1.15)[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
model = ft_model.merge_and_unload()

/home/tpllmws23/environments/finetuning/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
model.save_pretrained("merged_adapters")
#eval_tokenizer.pad_token = eval_tokenizer.unk_token
eval_tokenizer.save_pretrained("merged_adapters")

('merged_adapters/tokenizer_config.json',
 'merged_adapters/special_tokens_map.json',
 'merged_adapters/tokenizer.model',
 'merged_adapters/added_tokens.json',
 'merged_adapters/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
hub_id = "mistral-rag-instruct"
#model.save_pretrained(f"{hub_id}")
#eval_tokenizer.save_pretrained(f"{hub_id}")

print(f"Uploading to Hugging Face Hub: {hub_id}")
model.push_to_hub(f"{hub_id}")
eval_tokenizer.push_to_hub(f"{hub_id}")

Uploading to Hugging Face Hub: mistral-rag-instruct


model.safetensors:   0%|          | 0.00/4.41G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BenjaminBruenau/mistral-rag-instruct/commit/5d46b37ba2657fafa0abbc1cd5f56f3d1ab39a69', commit_message='Upload tokenizer', commit_description='', oid='5d46b37ba2657fafa0abbc1cd5f56f3d1ab39a69', pr_url=None, pr_revision=None, pr_num=None)

# Push Finetuned Model to Hub

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

# checkpoints/raft/checkpoint-425
base_model = AutoModelForCausalLM.from_pretrained("unload", torch_dtype=torch.bfloat16, device_map={"": "cpu"})
tokenizer = AutoTokenizer.from_pretrained("unload")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
hub_id = "mistral-rag-instruct"
#model.save_pretrained(f"{hub_id}")
#eval_tokenizer.save_pretrained(f"{hub_id}")

print(f"Uploading to Hugging Face Hub: {hub_id}")
base_model.push_to_hub(f"{hub_id}")
tokenizer.push_to_hub(f"{hub_id}")

Uploading to Hugging Face Hub: mistral-rag-instruct


model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/BenjaminBruenau/mistral-rag-instruct/commit/92ad810150eb971fe514b237d5773fa920e631c0', commit_message='Upload tokenizer', commit_description='', oid='92ad810150eb971fe514b237d5773fa920e631c0', pr_url=None, pr_revision=None, pr_num=None)

# Merge and Manual GGUF Conversion

In [ ]:
from unsloth import FastLanguageModel

#"checkpoints/raft-completion-only/checkpoint-350"
model, tokenizer = FastLanguageModel.from_pretrained("checkpoints/raft/checkpoint-425", load_in_4bit = True)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Unsloth unsuccessfully patched LoraLayer.update_layer. Please file a bug report.
Luckily, your training run will still work in the meantime!


==((====))==  Unsloth: Fast Mistral patching release 2024.6
   \\   /|    GPU: NVIDIA TITAN RTX. Max memory: 23.643 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: Will load checkpoints/raft/checkpoint-425 as a legacy tokenizer.
unsloth/mistral-7b-v0.3-bnb-4bit does not have a padding token! Will use pad_token = [control_768].
Unsloth 2024.6 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 179.01 out of 251.41 RAM for saving.


100%|██████████| 32/32 [00:01<00:00, 29.43it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


# TESTING

- we tried to dequantize and merge the model by hand to see if that mitigates the conversion quality loss

In [1]:
import torch
import peft
import json
import shutil
from peft.utils import _get_submodules
import os
import bitsandbytes as bnb
from bitsandbytes.functional import dequantize_4bit
from peft import PeftModel
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import gc
import copy

def dequantize_model(model, to='./dequantized_model', dtype=torch.float16, device="cuda"):
    """
    'model': the peftmodel you loaded with qlora.
    'tokenizer': the model's corresponding hf's tokenizer.
    'to': directory to save the dequantized model
    'dtype': dtype that the model was trained using
    'device': device to load the model to
    """


    os.makedirs(to, exist_ok=True)

    cls = bnb.nn.Linear4bit

    with torch.no_grad():
        for name, module in model.named_modules():
            if isinstance(module, cls):
                print(f"Dequantizing `{name}`...")
                quant_state = copy.deepcopy(module.weight.quant_state)
                quant_state.dtype = dtype

                weights = dequantize_4bit(module.weight.data, quant_state=quant_state, quant_type="nf4").to(dtype)

                new_module = torch.nn.Linear(module.in_features, module.out_features, bias=None, dtype=dtype)
                new_module.weight = torch.nn.Parameter(weights)
                new_module.to(device="cpu", dtype=dtype)

                parent, target, target_name = _get_submodules(model, name)
                setattr(parent, target_name, new_module)

        # a hack, setting this to avoid hf's saving error because hf
        # itself does not support saving a model that is registered to be loaded in 4bit.
        model.is_loaded_in_4bit = False

        print("Saving dequantized model...")
        model.save_pretrained(to)
        #tokenizer.save_pretrained(to)
        config_data = json.loads(open(os.path.join(to, 'config.json'), 'r').read())
        config_data.pop("quantization_config", None)
        config_data.pop("pretraining_tp", None)
        with open(os.path.join(to, 'config.json'), 'w') as config:
            config.write(json.dumps(config_data, indent=2))

        return model



In [2]:
model_name = "mistralai/Mistral-7B-v0.3"
adapter = "checkpoints/raft/checkpoint-425"

quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



#model = dequantize_model(model, to='./dqz_model/', dtype=torch.bfloat16)
'''
model =  AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map='cuda'
)
print(model)

model = PeftModel.from_pretrained(model, adapter)

print(model)

print(f"Successfully loaded the model {model_name} into memory")
model.save_pretrained("test", safe_serialization=True)


'''
try:
    print(f"Starting to load the model {model_name} into memory")

    model =  AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map={"": 0}
    )
    print(model)
    model = dequantize_model(model, to='./dqz_model/', dtype=torch.bfloat16)
    print(model)
    model = PeftModel.from_pretrained(model, adapter)
    print(model)
    model = model.merge_and_unload()
    print(model)

    print(f"Successfully loaded the model {model_name} into memory")
    model.save_pretrained("test", safe_serialization=True)
except Exception as e:
    print(f"An error occurred: {e}")

    # Delete the model object if it exists
    if 'model' in locals():
        del model

    # Clear the GPU cache
    torch.cuda.empty_cache()

    # Run the garbage collection
    gc.collect()

    print("Model, GPU cache, and garbage have been cleared.")


Starting to load the model mistralai/Mistral-7B-v0.3 into memory


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [3]:
# Free VRAM
del model
torch.cuda.empty_cache()
gc.collect()


17762